In [20]:
import numpy as np
import pandas as pd
from tensorflow import keras
import pandas as pd
from pandas import pivot_table as pivot
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from keras import Sequential

import keras
from IPython.display import SVG
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot

In [21]:
rating_data = pd.read_csv('rating_data.csv')

In [22]:
unqieid = set(rating_data.userid)
unqieid=list(unqieid)
userindex = dict((word,unqieid.index(word)+1) for word in unqieid)
rating_data['userid'].replace(userindex,inplace=True)
rating_data

,userid,itemid,rating
0,62,1,5
1,100,10,5
2,4,13,4
3,47,6,3
4,66,18,2
...,...,...,...
603,97,9,4
604,98,33,3
605,33,34,2
606,123,28,1


In [23]:
battery_data = pd.read_excel('battery.xlsx')
battery_data

,battery_id,battery_brand,volume(kwh),Price
0,1,Powerwall 2,13.50,1022
1,2,Sonnen ECO4,16.00,1220
2,3,Home V2,10.00,1176
3,4,Pulse 6,6.50,1236
4,5,BX6.3,6.30,1180
5,6,RESU Series1,3.30,710
6,7,RESU Series2,6.50,710
7,8,RESU Series3,9.80,710
8,9,RESU Series4,13.00,710
9,10,RESU H Series1,6.50,795


In [24]:
battery_name = list(battery_data['battery_brand'])

In [25]:
battery_name = dict((word,battery_name.index(word)+1) for word in battery_name)

In [26]:
battery_data['battery_brand'].replace(battery_name,inplace=True)

In [27]:
battery_data

,battery_id,battery_brand,volume(kwh),Price
0,1,1,13.50,1022
1,2,2,16.00,1220
2,3,3,10.00,1176
3,4,4,6.50,1236
4,5,5,6.30,1180
5,6,6,3.30,710
6,7,7,6.50,710
7,8,8,9.80,710
8,9,9,13.00,710
9,10,10,6.50,795


In [28]:
user_data = pd.read_csv('userdata.csv')
user_data
user_data['userid'].replace(userindex,inplace=True)

In [29]:
user_data

,userid,Energy conumption perday
0,62,24.90200
1,100,22.47300
2,4,5.97500
3,47,7.76000
4,66,10.16600
...,...,...
145,22,31.12700
146,3,144.70000
147,116,42.00000
148,6,32.29999


In [30]:
num_battery=39
num_user=150
n_latent_factor_user=5
n_latent_factor_item=5

In [31]:
item_input = keras.layers.Input(shape=[1],name='Item')
item_embedding = keras.layers.Embedding(num_battery + 1, n_latent_factor_user, name='Movie-Embedding')(item_input)
item_vec = keras.layers.Flatten(name='FlattenMovies')(item_embedding)
item_vec = keras.layers.Dropout(0.2)(item_vec)

In [32]:
user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(num_user + 1, n_latent_factor_user,name='User-Embedding')(user_input))
user_vec = keras.layers.Dropout(0.2)(user_vec)

In [33]:
concat = keras.layers.dot([item_vec,user_vec],axes=(1,1))
concat_dropout=keras.layers.Dropout(0.2)(concat)

In [34]:
dense=keras.layers.Dense(200)(concat)
dropout_1=keras.layers.Dropout(0.2)(dense)
dense_2= keras.layers.Dense(100)(concat)
dropout_2=keras.layers.Dropout(0.2)(dense_2)
dense_3=keras.layers.Dense(50)(dense_2)
dropout_3=keras.layers.Dropout(0.2)(dense_3)
dense_4=keras.layers.Dense(20,activation='relu')(dense_3)

In [35]:
result=keras.layers.Dense(1,activation='relu')(dense_4)
adam=Adam(lr=0.005)
model =keras.Model([user_input,item_input],result)
model.compile(optimizer=adam,loss='mean_absolute_error',metrics=['acc'])

In [36]:
data1 = pd.merge(rating_data,user_data)
data1

,userid,itemid,rating,Energy conumption perday
0,62,1,5,24.90200
1,62,9,5,24.90200
2,62,17,5,24.90200
3,62,39,5,24.90200
4,100,10,5,22.47300
...,...,...,...,...
603,6,23,4,32.29999
604,6,17,4,32.29999
605,99,8,3,88.11000
606,99,29,5,88.11000


In [37]:
data1.columns

Index(['userid', 'itemid', 'rating', 'Energy conumption perday'], dtype='object')

In [38]:
data1.columns=['user_id', 'battery_id', 'rating', 'Energy_conumption_perday']

In [39]:
data1

,user_id,battery_id,rating,Energy_conumption_perday
0,62,1,5,24.90200
1,62,9,5,24.90200
2,62,17,5,24.90200
3,62,39,5,24.90200
4,100,10,5,22.47300
...,...,...,...,...
603,6,23,4,32.29999
604,6,17,4,32.29999
605,99,8,3,88.11000
606,99,29,5,88.11000


In [40]:
final_data = pd.merge(data1,battery_data)

In [41]:
final_data

,user_id,battery_id,rating,Energy_conumption_perday,battery_brand,volume(kwh),Price
0,62,1,5,24.90200,1,13.5,1022
1,139,1,5,19.55600,1,13.5,1022
2,119,1,1,6.74100,1,13.5,1022
3,82,1,5,20.94300,1,13.5,1022
4,40,1,4,93.82000,1,13.5,1022
...,...,...,...,...,...,...,...
603,33,34,2,65.49999,34,8.3,800
604,50,34,4,64.97400,34,8.3,800
605,111,34,3,10.16800,34,8.3,800
606,10,34,3,57.10001,34,8.3,800


In [42]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(final_data, test_size=0.2)

In [43]:
train.head()

,user_id,battery_id,rating,Energy_conumption_perday,battery_brand,volume(kwh),Price
401,58,29,2,22.005,29,17.1,1114
36,32,9,1,5.245,9,13.0,710
16,130,1,4,35.800,1,13.5,1022
279,150,36,2,6.789,36,10.2,856
328,134,37,2,6.204,37,10.0,1950


In [ ]:
training = model.fit(x=[train[['user_id','Energy_conumption_perday']],train[['battery_id','battery_brand','volume(kwh)','Price']]],y=train.rating,epochs=250,verbose=2,validation_split=0.1)

In [ ]:
train[['user_id','Energy_conumption_perday']]

In [ ]:
train[['battery_id','battery_brand','volume(kwh)','Price']]